# Reprojecting vector objects

Introducing a point or a bounding box involves introducing a new library - `shapley`.  Shapely is a library that handles vector data, such as points and polygons, as opposed to `rasterio` the raster data that is used in other notebooks.

In [1]:
from IPython.display import Image

In [2]:
Image(url= "https://lh3.googleusercontent.com/proxy/LHbeQ132hF_CGGOSnP7tH12Hea26qfaGmiHFICPMA7emrKvXc-FnPCWXqqQgXbwm_HtLfMnrDRHY0Ub4WReyzhVvl--U71lz2-RSPJoUk5lUqe_ZsZEJ6lm_wAc7cftg3Gcq2tKhs2qobaCc", width=330)

### The Reproject Function

This code is a custom function that we are defining and calling `reproject`.  It uses some less-common functionality so I am not going to dig into the inside of this function.  For now just know that once you run this cell you can call `reproject()` as a function for the rest of the notebook.

In [3]:
import pyproj
from shapely import ops
from functools import partial

def reproject(geom, from_proj=None, to_proj=None):
    """
    Recommended function provided by shapely for projection transformations
    :param geom: shapely geometry object
    :param to_proj: source projection string
    :param from_proj: destination projection string
    :return: reprojected shapely geometry
    """
    tfm = partial(pyproj.transform, pyproj.Proj(from_proj), pyproj.Proj(to_proj))
    return ops.transform(tfm, geom)

## Reprojecting a Point or Bounding Box

**Step 1 - Define the shape we want to reproject**

Let's pick a shape to reproject.

I want to pick the bounding box of the raster as a polygon to reproject and the geometric center of our raster (the **centroid**) as a point.

In [4]:
import rasterio
from shapely.geometry import box

In [5]:
filepath = '../input_data/f150131t01p00r10_refl/f150131t01p00r10_h2o_v1'
with rasterio.open(filepath, 'r') as src:
    bbox = src.bounds

In [6]:
utm_bbox = box(*bbox)
utm_centroid = utm_bbox.centroid
print('Centroid: ', utm_centroid)
print('Bounding box: ', utm_bbox)

Centroid:  POINT (309240.5240943021 3892606.015206081)
Bounding box:  POLYGON ((339811.7357139058 3817906.830412161, 339811.7357139058 3967305.2, 278669.3124746986 3967305.2, 278669.3124746986 3817906.830412161, 339811.7357139058 3817906.830412161))


The object that is returned from from the `.centroid` method is a `shapely.geometry.point.Point` object.

Alternatively, we could use some other random point, but we will have to create that `Point` object ourselves.

In [7]:
from shapely.geometry import Point

In [8]:
utm_point = Point(500000, 0)

Notice that I chose a point which was at the center easting and sits on the equator (assuming a northern hemisphere grid).  We should be able to check that in our output.

**Step 2 - Define the input and output coordinate reference systems**

The source crs we can pull directly from the raster.  Since we want latitude and longitude as a result we can manually specify `EPSG:4326`.

In [9]:
with rasterio.open(filepath, 'r') as src:
    src_crs = src.crs
print('source crs is: ', src_crs)

source crs is:  EPSG:32610


In [10]:
dst_crs = 'EPSG:4326'
print('destination crs is: ', dst_crs)

destination crs is:  EPSG:4326


**Step 3 - Input the shape into our reproject function**

Once our point is defined we can input it into our reproject function.

In [11]:
wgs84_point = reproject(utm_point, from_proj=src_crs, to_proj=dst_crs)
wgs84_centroid = reproject(utm_centroid, from_proj=src_crs, to_proj=dst_crs)
wgs84_bbox = reproject(utm_bbox, from_proj=src_crs, to_proj=dst_crs)

In [12]:
print('random point: ', wgs84_point)
print('centroid: ', wgs84_centroid)
print('bounding box: ', wgs84_bbox)

random point:  POINT (0 -123)
centroid:  POINT (35.15829721247052 -125.0943601493459)
bounding box:  POLYGON ((34.49033534054283 -124.7446096774593, 35.83685310255475 -124.773613313877, 35.82496083874257 -125.4501069981971, 34.47901874476877 -125.4100634442505, 34.49033534054283 -124.7446096774593))


# Reprojecting a raster << Work in Progress >>

Reprojecting a raster is a bit more complicated than reprojecting vectors, but it is certainlly still doable.  The code I have below is largely based on the [rasterio docs](https://rasterio.readthedocs.io/en/latest/topics/reproject.html).

In [13]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [14]:
# Make the output directory if it does not exist yet
import os
if not os.path.exists('../output_data'):
    os.makedirs('../output_data')

In [15]:
dst_crs = 'EPSG:4326'

# Get the info we need from the existing raster
filepath_h2o = '../input_data/f150131t01p00r10_refl/f150131t01p00r10_h2o_v1'
with rasterio.open(filepath_h2o, 'r') as src:
    src_height, src_width = src.height, src.width
    src_crs, src_transform = src.crs, src.transform
    src_bounds = src.bounds
    metadata = src.meta.copy()
    bands = src.read()

# Calculate the new height, width, and affine transform
dst_transform, dst_width, dst_height = calculate_default_transform(src_crs, dst_crs, 
                                                                   src_width, src_height, *src_bounds)

# Update the metadata with the new info
metadata.update({
    'crs': dst_crs,
    'transform': dst_transform,
    'width': dst_width,
    'height': dst_height
})

# Create an empty raster for the output
reprojected = np.full(bands.shape, np.nan)

# Run the reprojection
reproject(
    source=bands,
    destination=reprojected,
    src_transform=src_transform,
    src_crs=src_crs,
    dst_transform=dst_transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest)

print(metadata)
# The array `reprojected` now has the output values and we can use that to save out the raster
with rasterio.open(
    '../output_data/reprojected',
    'w',
    **metadata
) as dst:
    dst.write(bands)

{'driver': 'ENVI', 'dtype': 'int16', 'nodata': None, 'width': 4142, 'height': 7972, 'count': 3, 'crs': 'EPSG:4326', 'transform': Affine(0.00017033327478689546, 0.0, -125.4501069981971,
       0.0, -0.00017033327478689546, 35.83685310255474)}


The exact copy of this code from the docs is actually this:

In [16]:
dst_crs = 'EPSG:4326'

filepath_h2o = '../input_data/f150131t01p00r10_refl/f150131t01p00r10_h2o_v1'
with rasterio.open(filepath_h2o) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    print(kwargs)
    with rasterio.open('../output_data/reprojected_h2o', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

{'driver': 'ENVI', 'dtype': 'int16', 'nodata': None, 'width': 4142, 'height': 7972, 'count': 3, 'crs': 'EPSG:4326', 'transform': Affine(0.00017033327478689546, 0.0, -125.4501069981971,
       0.0, -0.00017033327478689546, 35.83685310255474)}


This code is much more concise than what I have above but it might also be more difficult to follow because it consolidates multiple steps in the same line.